In [ ]:
# Imports
import pandas as pd
# import numpy as np
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import streamlit as st

In [ ]:
# read csv files
articles_data = pd.read_csv('./datasets/NIFTY_500_Articles.csv', index_col=0)
ticker_metadata = pd.read_csv('./datasets/ticker_metadata.csv', index_col=0)


In [ ]:
#Universe filter variable
universe = 'NIFTY_50'
universe_tickers = pd.read_csv('./datasets/{}.csv'.format(universe), index_col=0)['Symbol']

In [ ]:
#date and time filter variables
date_interval = 30
cutoff_date = datetime.datetime.now().date() - datetime.timedelta(date_interval)

In [ ]:
# filter articles based on date filter
date_filter_article = articles_data[pd.to_datetime(
    articles_data['Date']).dt.date > cutoff_date]
# calculate mean sentiment score for each ticker
filter_article_sent_score = date_filter_article.groupby(
    'Ticker').mean().reset_index()


In [ ]:
date_filter_article[date_filter_article['Ticker'] == 'SBIN'][['Headline','Date','Time']].reset_index(drop=True)

In [ ]:
filter_article_sent_score

In [ ]:
filter_article_sent_score[filter_article_sent_score['Ticker'] == 'HDFCLIFE']

In [ ]:
#filter companies based on universe
universe_filtered_df = filter_article_sent_score[filter_article_sent_score['Ticker'].isin(universe_tickers)]

In [ ]:
universe_filtered_df

In [ ]:
final_df = pd.merge(ticker_metadata, universe_filtered_df, on='Ticker', how='inner')

In [ ]:
final_df.columns = ['Symbol','Macro-Sector','Industry', 'Market Cap (Billion Rs)','Company Name','XC-Sector', 'XC-Industry', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']

In [ ]:
final_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
fig = px.treemap(
    final_df, path=[px.Constant('NSE'), 'XC-Sector', 'XC-Industry', 'Symbol'], values= 'Market Cap (Billion Rs)', color='Sentiment Score',
    hover_data=['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score'], color_continuous_scale=['#FF0000', "#000000", '#00FF00'], color_continuous_midpoint=0
    )
#fig.data[0].customdata = final_df[['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']]
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"
fig.update_traces(textposition="middle center")
fig.update_layout(height=800)
fig.update_layout(margin = dict(t=30, l=10, r=10, b=10), font_size=20)

In [ ]:
# Get current date, time and timezone to print to the App
now = datetime.datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
timezone_string = datetime.datetime.now().astimezone().tzname()

-----------------------------------

In [2]:
import investpy

In [13]:
investpy.indices.get_indices('india').symbol.to_list()

['SPBSEHC',
 'BSETECK',
 'SPBSFIIP',
 'SPBSEIT',
 'BSEMET',
 'BSEOIL',
 'BSEPOWER',
 'BSEPSU',
 'BSEREAL',
 'NIFVIX',
 'B50INCDN',
 'NIFTY200',
 'NSED',
 'NV20',
 'NIFTY500',
 'NIFMDCP100',
 'NIMDCP50',
 'NN50',
 'SPBSEFMCG',
 'NIFSMCP100',
 'NIFTY100',
 'BSECD',
 'NIFTYFMCG',
 'BSEBANK',
 'BKINXTR',
 'BSESN',
 'NSEI',
 'NIFTYAUTO',
 'NSEBANK',
 'NIFTYCOM',
 'NIFTYFIN',
 'NISM250',
 'NIFTYCONS',
 'NIFTYINFR',
 'NIFTYIT',
 'NIFTYMED',
 'NIFTYMET',
 'NIFTYMNC',
 'NIPHARM',
 'NIFTYPSU',
 'NIFTYREAL',
 'NIFTYSER',
 'BSEAUTO',
 'BSECG',
 'BKINX',
 'SPBSAIP',
 'SPBSESC',
 'BSECRBX',
 'SPBSEBH',
 'SPBS400M',
 'SPBS250S',
 'SPBS250L',
 'SPBS150M',
 'SPBSEDSI',
 'SPBSEESG',
 'DNIFTRI',
 'NIFTYPSE',
 'NIFML15',
 'NIFTYLVI',
 'NIFTYHBI',
 'NI15',
 'NIFTRI',
 'SPBSEEVI',
 'BSEGRNX',
 'BSEIINF',
 'MIN0000TPUS',
 'BSEDX30',
 'BSEDX200',
 'BSEDX100',
 'BSECPSE',
 'BSESMEIPO',
 'SPBSSCSI',
 'SPBSESNXT50',
 'SPBSESN50',
 'SPBSEQLI',
 'SPBSEMOI',
 'SPBSMCSI',
 'SPBSELVI',
 'BSEIPO',
 'SPBIMIP',
 'NIFTYD

In [3]:
import pandas as pd


In [4]:
pd.read_csv('https://archives.nseindia.com/content/indices/ind_nifty50list.csv')

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
1,Adani Ports and Special Economic Zone Ltd.,Services,ADANIPORTS,EQ,INE742F01042
2,Apollo Hospitals Enterprise Ltd.,Healthcare,APOLLOHOSP,EQ,INE437A01024
3,Asian Paints Ltd.,Consumer Durables,ASIANPAINT,EQ,INE021A01026
4,Axis Bank Ltd.,Financial Services,AXISBANK,EQ,INE238A01034
5,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO,EQ,INE917I01010
6,Bajaj Finance Ltd.,Financial Services,BAJFINANCE,EQ,INE296A01024
7,Bajaj Finserv Ltd.,Financial Services,BAJAJFINSV,EQ,INE918I01026
8,Bharat Petroleum Corporation Ltd.,Oil Gas & Consumable Fuels,BPCL,EQ,INE029A01011
9,Bharti Airtel Ltd.,Telecommunication,BHARTIARTL,EQ,INE397D01024


In [2]:
import nsepython as npy

In [4]:
npy.nse_eq('M&M')

{'info': {'symbol': 'M&M',
  'companyName': 'Mahindra & Mahindra Limited',
  'industry': 'AUTOMOBILES - 4 WHEELERS',
  'activeSeries': ['EQ'],
  'debtSeries': [],
  'tempSuspendedSeries': [],
  'isFNOSec': True,
  'isCASec': False,
  'isSLBSec': True,
  'isDebtSec': False,
  'isSuspended': False,
  'isETFSec': False,
  'isDelisted': False,
  'isin': 'INE101A01026',
  'isTop10': False,
  'identifier': 'M&MEQN'},
 'metadata': {'series': 'EQ',
  'symbol': 'M&M',
  'isin': 'INE101A01026',
  'status': 'Listed',
  'listingDate': '03-Jan-1996',
  'industry': 'Passenger Cars & Utility Vehicles',
  'lastUpdateTime': '31-May-2023 16:00:00',
  'pdSectorPe': 28.83,
  'pdSymbolPe': 25.04,
  'pdSectorInd': 'NIFTY AUTO                                        '},
 'securityInfo': {'boardStatus': 'Main',
  'tradingStatus': 'Active',
  'tradingSegment': 'Normal Market',
  'sessionNo': '-',
  'slb': 'Yes',
  'classOfShare': 'Equity',
  'derivatives': 'Yes',
  'surveillance': {'surv': None, 'desc': None},


In [10]:
symbol = 'L&TFH'

In [11]:
symbol.replace('&', '%26')

'L%26TFH'

In [1]:
!pip3 list --outdated

Package           Version   Latest   Type
----------------- --------- -------- -----
beautifulsoup4    4.12.1    4.12.2   wheel
certifi           2022.12.7 2023.5.7 wheel
fonttools         4.39.3    4.39.4   wheel
fsspec            2023.4.0  2023.5.0 wheel
ipykernel         6.22.0    6.23.1   wheel
ipython           8.12.0    8.13.2   wheel
nsepython         0.0.973   1.5      wheel
numpy             1.24.2    1.24.3   wheel
pandas            1.5.3     2.0.2    wheel
platformdirs      3.2.0     3.5.1    wheel
pyzmq             25.0.2    25.1.0   wheel
regex             2023.3.23 2023.5.5 wheel
requests          2.28.2    2.31.0   wheel
setuptools        67.6.1    67.8.0   wheel
tornado           6.3.1     6.3.2    wheel
transformers      4.28.1    4.29.2   wheel
typing_extensions 4.5.0     4.6.2    wheel
urllib3           1.26.15   2.0.2    wheel


In [6]:
tickers = ['M&M', 'SBIN', 'HDFC']

In [8]:
dict = {
    'M&M': 'xyz122'
}

In [7]:
url = 'ticker.finology.com/company'

In [10]:
for ticker in tickers:
    print(dict[ticker]) if ticker in dict.keys() else print(ticker)

xyz122
SBIN
HDFC


In [15]:
tuple(zip('a', '0'))

(('a', '0'),)

In [17]:
import pandas as pd

In [19]:
nifty_500_ticker_url = 'https://archives.nseindia.com/content/indices/ind_nifty500list.csv'
nifty_500 = pd.read_csv(
    'https://archives.nseindia.com/content/indices/ind_nifty500list.csv')
